# 2D indentation of a fibre in a matrix



In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import matplotlib as mpl
import hardness as hd
import argiope as ag
import pandas as pd
import numpy as np
import os, subprocess, time, local_settings, time


mpl.rcParams["grid.color"] = "k"
mpl.rcParams["grid.linestyle"] = ":"
mpl.rcParams["grid.linewidth"] = 0.5
mpl.rcParams["contour.negative_linestyle"] = "solid"


# USEFUL FUNCTIONS
def create_dir(path):
    try:
        os.mkdir(path)
    except:
        pass

## Settings

In [ ]:
# SETTINGS
workdir = "workdir/"
outputdir = "outputs/"
label = "indentation_2D"

create_dir(workdir)
create_dir(workdir + outputdir)

## Model definition

In [ ]:
# -------------------------------------------------------------------------------
# MESH DEFINITIONS
def element_map(mesh):
    mesh.elements.loc[
        mesh.elements.type.argiope == "tri3", ("type", "solver", "")
    ] = "CAX3"
    mesh.elements.loc[
        mesh.elements.type.argiope == "quad4", ("type", "solver", "")
    ] = "CAX4"
    return mesh


def sample_material_map(mesh):
    mesh.elements.loc[mesh.elements.sets.FIBRE, "materials"] = "FIBRE_MAT"
    mesh.elements.loc[mesh.elements.sets.MATRIX, "materials"] = "MATRIX_MAT"
    return mesh


def indenter_material_map(mesh):
    mesh.elements["materials"] = "INDENTER_MAT"
    return mesh


parts = {
    "sample": hd.models.SampleFibre2D(
        Rf=1.0,
        ly1=1.0,
        ly2=5.0,
        Nx=16,
        Ny=8,
        Nr=8,
        Nt=None,
        gmsh_path="gmsh",
        file_name="dummy",
        workdir=workdir,
        gmsh_space=2,
        gmsh_options="-algo 'delquad'",
        element_map=element_map,
        material_map=sample_material_map,
    ),
    "indenter": hd.models.SpheroconicalIndenter2D(
        R=1.0,
        psi=70.3,
        r1=1.0,
        r2=3.0,
        r3=3.0,
        lc1=0.1,
        lc2=0.5,
        rigid=False,
        gmsh_path="gmsh",
        file_name="dummy",
        workdir=workdir,
        gmsh_space=2,
        gmsh_options="-algo 'delquad'",
        element_map=element_map,
        material_map=indenter_material_map,
    ),
}

materials = [
    ag.materials.ElasticPerfectlyPlastic(
        label="MATRIX_MAT", young_modulus=0.1, poisson_ratio=0.3, yield_stress=0.001
    ),
    ag.materials.Elastic(label="INDENTER_MAT", young_modulus=1.0, poisson_ratio=0.3),
    ag.materials.Elastic(label="FIBRE_MAT", young_modulus=0.4, poisson_ratio=0.3),
]

# -------------------------------------------------------------------------------
# STEP DEFINTIONS
steps = [
    hd.models.Step2D(
        name="LOADING1",
        control_type="disp",
        duration=1.0,
        kind="adaptative",
        nframes=100,
        controlled_value=-0.2,
        field_output_frequency=99999,
    ),
    hd.models.Step2D(
        name="UNLOADING1",
        control_type="force",
        duration=1.0,
        kind="adaptative",
        nframes=100,
        controlled_value=0.0,
        field_output_frequency=99999,
    ),
    hd.models.Step2D(
        name="RELOADING1",
        control_type="disp",
        duration=1.0,
        kind="adaptative",
        nframes=100,
        controlled_value=-0.2,
        field_output_frequency=99999,
    ),
    hd.models.Step2D(
        name="LOADING2",
        control_type="disp",
        duration=1.0,
        kind="adaptative",
        nframes=100,
        controlled_value=-0.4,
        field_output_frequency=99999,
    ),
    hd.models.Step2D(
        name="UNLOADING2",
        control_type="force",
        kind="adaptative",
        duration=1.0,
        nframes=50,
        controlled_value=0.0,
        field_output_frequency=99999,
    ),
]

model0 = hd.models.Indentation2D(
    label=label,
    parts=parts,
    steps=steps,
    materials=materials,
    solver="abaqus",
    solver_path=local_settings.ABAQUS_PATH,
    workdir=workdir,
    verbose=True,
)

In [ ]:
print("1: Preprocessing ----------------------------------")
%time model0.write_input()
print("2: Processing -------------------------------------")
%time model0.run_simulation()
print("3: Postprocessing ---------------------------------")
%time model0.postproc()
print("4: Saving model -----------------------------------")
%time model0.save(workdir + "model.pcklz")

In [ ]:
model = ag.utils.load(workdir + "model.pcklz")

## Model checking

Mesh building and quality checking.

In [ ]:
model.parts["indenter"].mesh.elements.head()

In [ ]:
model.parts["sample"].mesh.elements.head()

In [ ]:
parts = model.parts
i = 1
fig = plt.figure()
parts_names = parts.keys()
for name, part in parts.items():
    mesh = part.mesh
    patches = mesh.to_polycollection(edgecolor="black", linewidth=0.5, alpha=1.0)
    stats = mesh.stats()
    patches.set_array(stats.stats.max_abs_angular_deviation)
    patches.set_cmap(mpl.cm.YlOrRd)
    ax = fig.add_subplot(1, 2, i)
    ax.set_aspect("equal")
    ax.set_xlim(mesh.nodes.coords.x.min(), mesh.nodes.coords.x.max())
    ax.set_ylim(mesh.nodes.coords.y.min(), mesh.nodes.coords.y.max())
    ax.add_collection(patches)
    cbar = plt.colorbar(patches, orientation="horizontal")
    cbar.set_label("Max Abs. Angular Deviation [$^o$]")
    plt.xlabel("$x$")
    plt.ylabel("$y$")
    plt.grid()
    plt.title(name.title())
    i += 1
plt.show()

## Simulation

## Post-Processing

### Time data


In [ ]:
hist = model.data["history"]
hist.head()

In [ ]:
plt.figure()
for step, group in hist.groupby("step"):
    plt.plot(group.disp.htot, group.force.F, label="Step {0}".format(step))
plt.grid()
plt.legend(loc="best")
plt.ylabel("Total force $F$, []")
plt.xlabel("Displacement, $\delta$ []")
plt.show()

### Fields

In [ ]:
model.parts["sample"].mesh.fields_metadata()

In [ ]:
model.parts["sample"].mesh.fields_metadata()

In [ ]:
parts = {k: part.mesh.copy() for k, part in model.parts.items()}

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.set_aspect("equal")
ax.set_xlim(0.0, 3.0)
ax.set_ylim(-2.0, 2.0)

field_num = 14
disp_num = 15
levels = np.linspace(-1.0e-1, 1.0e-1, 11)

for k, mesh in parts.items():
    field = mesh.fields[field_num].data.v22
    disp = mesh.fields[disp_num].data
    mesh.nodes[("coords", "x")] += disp.v1
    mesh.nodes[("coords", "y")] += disp.v2
    tri = mesh.to_triangulation()
    patches = mesh.to_polycollection(facecolor="none", edgecolor="black", linewidth=0.5)
    grad = ax.tricontourf(tri, field, levels, cmap=mpl.cm.terrain, alpha=1)
    ax.tricontour(tri, field, levels, colors="white", linewidths=1.0)
    ax.add_collection(patches)
cbar = plt.colorbar(grad)
cbar.set_label("Cauchy Stress, $\sigma_{12}$")
plt.xlabel("$x$")
plt.ylabel("$y$")
# plt.grid()

In [ ]:
parts["indenter"].fields

In [ ]:
mesh = parts["sample"]
tag = "SURFACE"

nodes = mesh.nodes.copy()
dummy = nodes.iloc[0].copy()
dummy["coords"] *= np.nan
dummy["sets"] = True
nodes.loc[0] = dummy
# Getting element surfaces
element_surfaces = mesh.split("surfaces").unstack()
# killer hack !
surf = (
    pd.DataFrame(
        nodes.sets[tag]
        .loc[element_surfaces.values.flatten()]
        .values.reshape(element_surfaces.shape)
        .prod(axis=1)
        .astype(np.bool_),
        index=element_surfaces.index,
    )
    .unstack()
    .astype(float)
    .fillna(0)
    .astype(bool)
)  # .fillna(False)
for k in surf.keys():
    mesh.elements["surfaces", tag, "f{0}".format(k[1] + 1)] = surf.loc[:, k]

mesh

In [ ]:
surf.astype(float).fillna(0).astype(bool)